In [1]:
import pandas as pd
import glob
import os
import json
import numpy as np
import inspect
import sys
import math

##### We will be saving all the final data in the df_final DF

In [2]:
df_pp = pd.read_csv('../data/movez_metadata/Participant_Info.csv', sep=';', header=0)
df_final = df_pp[['Child']]
df_final

,Child
0,902
1,904
2,907
3,908
4,909
...,...
1479,6180
1480,6181
1481,6182
1482,6183


In [4]:
df_physical = pd.read_csv('../data/movez_health_data/Fitbit_W17_cleaned_imputation.csv', sep=';', header=0)
df_physical

,School,Class,Child,Sex,Age,Wave,Day,Date,Participated,Steps,Minutes_MVPA,Imputed
0,22,52,902,1,13.0,1,2,29-1-2016,1.0,15921.0,124.0,imputed
1,22,52,902,1,13.0,1,1,28-1-2016,1.0,3426.0,0.0,imputed
2,22,52,902,1,13.0,1,3,30-1-2016,1.0,12311.0,237.0,imputed
3,22,52,902,1,13.0,2,3,2-4-2016,1.0,23583.0,17.0,imputed
4,22,52,902,1,13.0,2,2,1-4-2016,1.0,1000.0,0.0,imputed
...,...,...,...,...,...,...,...,...,...,...,...,...
26461,59,307,6131,0,16.0,7,3,20-5-2018,1.0,17586.0,34.0,imputed
26462,59,307,6131,0,16.0,7,2,19-5-2018,1.0,8891.0,30.0,imputed
26463,59,307,6133,0,15.0,7,3,20-5-2018,1.0,14679.0,41.0,imputed
26464,59,307,6133,0,15.0,7,1,18-5-2018,1.0,1187.0,9.0,imputed


In [7]:
df_physical = df_physical.groupby('Child')['Steps','Minutes_MVPA'].mean().reset_index()
df_physical

/var/folders/t4/v59wrn6n2ws6kl1t9zh1q_mc0000gn/T/ipykernel_3559/2461543173.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_physical = df_physical.groupby('Child')['Steps','Minutes_MVPA'].mean().reset_index()


,Child,Steps,Minutes_MVPA
0,902,10351.750000,64.750000
1,904,10587.857143,45.571429
2,907,5270.153846,11.846154
3,908,8807.733333,14.333333
4,909,9146.529412,49.529412
...,...,...,...
1478,6180,7867.538462,13.538462
1479,6181,7748.076923,13.846154
1480,6182,7414.000000,30.000000
1481,6183,9662.916667,26.916667


In [20]:
df_bmi = pd.read_csv('../data/movez_health_data/BMI_final.csv', sep=';', header=0)
df_bmi

,Child,Wave,Sex,Age,Length,Weight,BMI,zBMI,BMIcor
0,902,2,1,13,NaN,NaN,NaN,NaN,NaN
1,904,2,1,12,NaN,NaN,NaN,NaN,NaN
2,907,2,1,12,NaN,NaN,NaN,NaN,NaN
3,908,2,0,12,"165,6","75,2","27,4218768232631","2,35351262491133",5.0
4,909,2,0,12,"150,8","54,5","23,9659042137776","1,80149175782306",4.0
...,...,...,...,...,...,...,...,...,...
4447,6180,5,1,12,"151,4","37,2","16,228978673726","-0,887682510437909",3.0
4448,6181,5,1,12,"156,8","40,7","16,553974906289","-0,724477099503862",3.0
4449,6182,5,0,12,"155,1","52,9","21,9903466950671","1,37615262312107",4.0
4450,6183,5,1,12,"155,6","39,8","16,4385643763919","-0,781546769566859",3.0


In [21]:
df_bmi = df_bmi.replace({',': '.'}, regex=True)

In [22]:
df_bmi.BMI = df_bmi.BMI.astype('float')

In [23]:
df_bmi = df_bmi.groupby('Child')['BMI'].mean().reset_index()
df_bmi

,Child,BMI
0,902,NaN
1,904,NaN
2,907,NaN
3,908,28.814745
4,909,24.660951
...,...,...
1479,6180,16.228979
1480,6181,16.553975
1481,6182,21.990347
1482,6183,16.438564


In [36]:
df_final['Steps'] = df_final['Child'].map(df_physical.set_index('Child')['Steps'])
df_final['Minutes_MVPA'] = df_final['Child'].map(df_physical.set_index('Child')['Minutes_MVPA'])
df_final['BMI'] = df_final['Child'].map(df_bmi.set_index('Child')['BMI'])
df_final

/var/folders/t4/v59wrn6n2ws6kl1t9zh1q_mc0000gn/T/ipykernel_3559/195405995.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Steps'] = df_final['Child'].map(df_physical.set_index('Child')['Steps'])
/var/folders/t4/v59wrn6n2ws6kl1t9zh1q_mc0000gn/T/ipykernel_3559/195405995.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Minutes_MVPA'] = df_final['Child'].map(df_physical.set_index('Child')['Minutes_MVPA'])
/var/folders/t4/v59wrn6n2ws6kl1t9zh1q_mc0000gn/T/ipykernel_3559/195405995.py:3: 

,Child,Steps,Minutes_MVPA,BMI
0,902,10351.750000,64.750000,NaN
1,904,10587.857143,45.571429,NaN
2,907,5270.153846,11.846154,NaN
3,908,8807.733333,14.333333,28.814745
4,909,9146.529412,49.529412,24.660951
...,...,...,...,...
1479,6180,7867.538462,13.538462,16.228979
1480,6181,7748.076923,13.846154,16.553975
1481,6182,7414.000000,30.000000,21.990347
1482,6183,9662.916667,26.916667,16.438564


In [37]:
df_final['Steps'] = df_final['Steps'].apply(np.round)
df_final['Minutes_MVPA'] = df_final['Minutes_MVPA'].apply(np.round)
df_final['BMI'] =df_final['BMI'].apply(np.round)
df_final

/var/folders/t4/v59wrn6n2ws6kl1t9zh1q_mc0000gn/T/ipykernel_3559/4235774141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Steps'] = df_final['Steps'].apply(np.round)
/var/folders/t4/v59wrn6n2ws6kl1t9zh1q_mc0000gn/T/ipykernel_3559/4235774141.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Minutes_MVPA'] = df_final['Minutes_MVPA'].apply(np.round)
/var/folders/t4/v59wrn6n2ws6kl1t9zh1q_mc0000gn/T/ipykernel_3559/4235774141.py:3: SettingWithCopyWarning: 
A value is trying to be set on

,Child,Steps,Minutes_MVPA,BMI
0,902,10352.0,65.0,NaN
1,904,10588.0,46.0,NaN
2,907,5270.0,12.0,NaN
3,908,8808.0,14.0,29.0
4,909,9147.0,50.0,25.0
...,...,...,...,...
1479,6180,7868.0,14.0,16.0
1480,6181,7748.0,14.0,17.0
1481,6182,7414.0,30.0,22.0
1482,6183,9663.0,27.0,16.0


In [38]:
df_final.to_csv('../output/health_variables.csv',index=False)